# Task 4: Portfolio Optimization Based on Forecasts

### This notebook will:

1. Combine historical and forecasted data for each asset (TSLA, BND, SPY).
2. Compute expected returns and volatility for each asset based on forecasted prices.
3. Optimize the portfolio allocation to maximize the Sharpe Ratio (risk-adjusted return).
4. Analyze the optimized portfolio’s risk, returns, and volatility, and visualize the cumulative performance.


### Import necessary libraries

In [1]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

### Define paths to data and models

In [2]:
# Define paths to data and models
os.chdir(r'c:\users\ermias.tadesse\10x\Predictive-Analysis-And-Modeling')
data_folder_path = 'Data/'
models_folder_path = 'Models/'